In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import io
import openox as ox
from exclude_unclean import *
from session_functions import threesamples, recalculate_so2_range, sample_stability, apply_sample_stability, assign_marker_color, assign_marker_style
from importlib import reload
import os


In [2]:
abg = pd.read_csv('../OpenOxPHIData/output_internal_bloodgas.csv')
abg = abg[['date','time_calc','sample','so2','session']]


In [40]:
base_path = '../OpenOxPHIData/waveforms/labview_2hz/'
pattern = 'labview_session_{}_2hz.csv'
cols_to_summarize = ['Masimo 97/SpO2','Nellcor/SpO2'] 
labview_files = ox.get_labview_files(abg, base_path, pattern)
labview_samples = ox.extract_values_by_sample(labview_files, 'session','Sample',5,cols_to_summarize,abg)

c:\Users\LawT\Downloads\python\openox\lipnick_session_streamlit\openox\__init__.py:273: RuntimeWarning: invalid value encountered in cast
  for i in abg['session'].unique().astype(int):


No file for session 139 at ../OpenOxPHIData/waveforms/labview_2hz/labview_session_139_2hz.csv
No file for session -2147483648 at ../OpenOxPHIData/waveforms/labview_2hz/labview_session_-2147483648_2hz.csv
No file for session 17 at ../OpenOxPHIData/waveforms/labview_2hz/labview_session_17_2hz.csv
No file for session 74 at ../OpenOxPHIData/waveforms/labview_2hz/labview_session_74_2hz.csv
No file for session 412 at ../OpenOxPHIData/waveforms/labview_2hz/labview_session_412_2hz.csv
No file for session 417 at ../OpenOxPHIData/waveforms/labview_2hz/labview_session_417_2hz.csv
extracting labview values.....


In [41]:
#set everything to keep to start
labview_samples['manual_clean_so2'] = 'keep'
labview_samples['manual_clean_masimo'] = 'keep'
labview_samples.head()

,session,sample,Masimo 97/SpO2,Nellcor/SpO2,so2,Timestamp,manual_clean_so2,manual_clean_masimo
0,13,1,100.0,100.000000,99.4,09:33:02,keep,keep
1,13,2,100.0,100.000000,98.9,09:34:06,keep,keep
2,13,3,100.0,98.090909,98.6,09:34:46,keep,keep
3,13,4,100.0,99.818182,99.0,09:35:19,keep,keep
4,13,5,100.0,100.000000,98.8,09:35:56,keep,keep


In [42]:
reload(ox)
labview_samples, counts = ox.sample_stability_multi(labview_samples, 'so2','Nellcor/SpO2', 'Timestamp','algo',1.5,2)
labview_samples

c:\Users\LawT\Downloads\python\openox\lipnick_session_streamlit\openox\__init__.py:350: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce')


,session,sample,Masimo 97/SpO2,Nellcor/SpO2,so2,Timestamp,manual_clean_so2,manual_clean_masimo,sample_diff_prev,sample_diff_next,...,so2_diff_next,Nellcor/SpO2_diff_prev,Nellcor/SpO2_diff_next,Timestamp_diff_prev,Timestamp_diff_next,so2_stable,so2_reason,Nellcor_stable,Nellcor_reason,algo
0,13,1,100.000000,100.000000,99.40,2024-05-09 09:33:02,keep,keep,0.0,-1.0,...,0.5,0.000000,0.000000,0.0,-64.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
1,13,2,100.000000,100.000000,98.90,2024-05-09 09:34:06,keep,keep,1.0,-1.0,...,0.3,0.000000,1.909091,64.0,-40.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
2,13,3,100.000000,98.090909,98.60,2024-05-09 09:34:46,keep,keep,1.0,-1.0,...,-0.4,-1.909091,-1.727273,40.0,-33.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
3,13,4,100.000000,99.818182,99.00,2024-05-09 09:35:19,keep,keep,1.0,-1.0,...,0.2,1.727273,-0.181818,33.0,-37.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
4,13,5,100.000000,100.000000,98.80,2024-05-09 09:35:56,keep,keep,1.0,-1.0,...,5.3,0.181818,3.909091,37.0,-389.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7803,548,21,89.181818,88.909091,87.65,2024-05-09 15:43:36,keep,keep,1.0,-1.0,...,2.5,0.909091,2.909091,29.0,-197.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
7804,548,22,87.000000,86.000000,85.15,2024-05-09 15:46:53,keep,keep,1.0,-1.0,...,0.9,-2.909091,0.000000,197.0,-24.0,True,keep because next sample is within bound,True,keep because next sample is within bound,keep
7805,548,23,87.000000,86.000000,84.25,2024-05-09 15:47:17,keep,keep,1.0,-1.0,...,2.1,0.000000,1.545455,24.0,-25.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
7806,548,24,85.000000,84.454545,82.15,2024-05-09 15:47:42,keep,keep,1.0,-1.0,...,-1.5,-1.545455,-0.545455,25.0,-28.0,False,reject because both previous and next samples ...,True,keep because previous sample is within bound,reject_so2


In [46]:
sensitivity_analysis = {}

for so2bound in range(1,3,.25):
    for refbound in range (1,3,.25):
        df, value_counts = ox.sample_stability_multi(labview_samples, 'so2','Nellcor/SpO2', 'Timestamp','algo',so2bound,refbound)
        sensitivity_analysis[(so2bound,refbound)] = value_counts

sensitivity_analysis 

TypeError: 'float' object cannot be interpreted as an integer

## Mark manually cleaned items from "drop dict" as 'rejected' in the manual clean columns

In [21]:
dothis = True

if dothis:
    for session in drop_dict:
        for sample in drop_dict[session]:
            labview_samples.loc[(labview_samples['session']==session) & (labview_samples['sample']==sample), 'manual_clean_so2'] = 'reject'

    for session in loc_dict:
        #access just the first item of the tuple
        for sample in loc_dict[session][0]:
            labview_samples.loc[(labview_samples['session']==session) & (labview_samples['sample']==sample), 'manual_clean_masimo'] = 'reject'

In [24]:
labview_samples

,session,sample,Masimo 97/SpO2,Nellcor/SpO2,so2,Timestamp,manual_clean_so2,manual_clean_masimo,sample_diff_prev,sample_diff_next,...,so2_diff_next,Nellcor/SpO2_diff_prev,Nellcor/SpO2_diff_next,Timestamp_diff_prev,Timestamp_diff_next,so2_stable,so2_reason,Nellcor_stable,Nellcor_reason,algo
0,13,1,100.000000,100.000000,99.40,2024-05-09 09:33:02,keep,keep,0.0,-1.0,...,0.5,0.000000,0.000000,0.0,-64.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
1,13,2,100.000000,100.000000,98.90,2024-05-09 09:34:06,keep,keep,1.0,-1.0,...,0.3,0.000000,1.909091,64.0,-40.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
2,13,3,100.000000,98.090909,98.60,2024-05-09 09:34:46,keep,keep,1.0,-1.0,...,-0.4,-1.909091,-1.727273,40.0,-33.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
3,13,4,100.000000,99.818182,99.00,2024-05-09 09:35:19,keep,keep,1.0,-1.0,...,0.2,1.727273,-0.181818,33.0,-37.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
4,13,5,100.000000,100.000000,98.80,2024-05-09 09:35:56,keep,keep,1.0,-1.0,...,5.3,0.181818,3.909091,37.0,-389.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7803,548,21,89.181818,88.909091,87.65,2024-05-09 15:43:36,keep,keep,1.0,-1.0,...,2.5,0.909091,2.909091,29.0,-197.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
7804,548,22,87.000000,86.000000,85.15,2024-05-09 15:46:53,keep,keep,1.0,-1.0,...,0.9,-2.909091,0.000000,197.0,-24.0,True,keep because next sample is within bound,True,keep because next sample is within bound,keep
7805,548,23,87.000000,86.000000,84.25,2024-05-09 15:47:17,keep,keep,1.0,-1.0,...,2.1,0.000000,1.545455,24.0,-25.0,True,keep because previous sample is within bound,True,keep because previous sample is within bound,keep
7806,548,24,85.000000,84.454545,82.15,2024-05-09 15:47:42,keep,keep,1.0,-1.0,...,-1.5,-1.545455,-0.545455,25.0,-28.0,False,reject because both previous and next samples ...,True,keep because previous sample is within bound,reject


In [7]:
labview_samples_post_algorithm = labview_samples.copy()
labview_samples_post_algorithm

,session,sample,Masimo 97/SpO2,Nellcor/SpO2,so2,Timestamp,manual_clean_so2,manual_clean_masimo
0,13,1,100.000000,100.000000,99.40,09:33:02,keep,keep
1,13,2,100.000000,100.000000,98.90,09:34:06,keep,keep
2,13,3,100.000000,98.090909,98.60,09:34:46,keep,keep
3,13,4,100.000000,99.818182,99.00,09:35:19,keep,keep
4,13,5,100.000000,100.000000,98.80,09:35:56,keep,keep
...,...,...,...,...,...,...,...,...
7803,548,21,89.181818,88.909091,87.65,15:43:36,keep,keep
7804,548,22,87.000000,86.000000,85.15,15:46:53,keep,keep
7805,548,23,87.000000,86.000000,84.25,15:47:17,keep,keep
7806,548,24,85.000000,84.454545,82.15,15:47:42,keep,keep


Do some calculations on the rejected and kept samples

In [10]:
so2mask = labview_samples['so2_keep'] != labview_samples['manual_clean_so2']
masimomask = labview_samples['masimo_keep'] != labview_samples['manual_clean_masimo']
manualmask = labview_samples['manual_clean_so2'] == 'reject'
algomask = labview_samples['so2_keep'] == 'reject'
manualrjecets = set(labview_samples[manualmask].index)
algorejects = set(labview_samples[algomask].index)


print('numnber of samples rejected by manual cleaning: ', labview_samples[manualmask].shape[0])
print('number of samples rejected by algorithm: ', labview_samples[algomask].shape[0],'\n')
print('# samples rejected by both:', len(manualrjecets.intersection(algorejects)))
print('# samples rejected by manual but not algorithm:', len(manualrjecets.difference(algorejects)))
print('# samples rejected by algorithm but not manual:', len(algorejects.difference(manualrjecets)),'\n')

print(labview_samples['so2_keep'].value_counts(normalize=True),'\n')
print(labview_samples['manual_clean_so2'].value_counts(normalize=True),'\n')


KeyError: 'so2_keep'